In [1]:
import pandas as pd
import math # Use math.isnan()

In [2]:
from useLangTool import *

useLangTool.test_word : 6.469999789260328e-07


# From google sheet to M2

In [3]:
glink = "./GEC - Our datasets(wiki) - Sheet1.tsv"
outfilename = "OurDataset.m2"
ourtestfilename = "OurDataset_test.txt"
ourgoldfilename = "OurDataset_gold.txt"

df = pd.read_csv(glink, sep="\t", encoding="utf8")
df = df[0:120]

with open(outfilename, mode="w", encoding="utf8") as f:
    for idx in range(0, len(df)):
        row = list(df.iloc[idx])
        incorr = row[1].replace("’", "'").replace("'", " '").replace(".", " . ").replace(", ", " , ").replace("!", " ! ").replace("  ", " ").replace("?", " ? ").strip("\n\r\t ") # .replace("’s ", " ’s ")
        corr = row[2]
        wc = row[3]

        #print(f"S {incorr}")
        f.write(f"S {incorr}\n")

        edits = row[4:16]
        for edit_idx in range(len(edits)):
            if 0 == edit_idx % 2:
                if "nan" != str(edits[edit_idx]):
                    errtype = edits[edit_idx]
                    chg = edits[edit_idx + 1]
                    
                    try:
                        pos = chg.split('|||')[0]
                        repl = chg.split('|||')[1]
                    except Exception as e:
                        continue

                    #print(f"A {' '.join(pos.split(','))}|||{errtype}|||{repl}|||REQUIRED|||-NONE-|||0")
                    f.write(f"A {' '.join(pos.split(','))}|||{errtype}|||{repl}|||REQUIRED|||-NONE-|||0\n")
        #print()
        f.write(f"\n")
        
        
#===========
arr_incorr = []
arr_corr = []

with open(outfilename, mode="r", encoding="utf8") as f:
    arr_rows = f.read().split("\n\n")
    for row in arr_rows:
        tuple_tmp = correct_sent(row)
        
        arr_incorr.append(tuple_tmp[0])
        arr_corr.append(tuple_tmp[1])
#===========
for item in list(zip(list(df["corr_sentence"]), arr_corr)):
    item0 = item[0].replace("’", "'").replace("'", " '").replace(".", " . ").replace(", ", " , ").replace("!", " ! ").replace("?", " ? ").replace("  ", " ").strip("\n\r\t ")
    if item0 != item[1]:
        # .replace("’s ", " ’s ")
        print(item0)
        print(item[1])
        print()

In [4]:
with open(ourtestfilename, mode="w", encoding="utf8") as f:
    f.write('\n'.join(arr_incorr))

In [5]:
with open(ourgoldfilename, mode="w", encoding="utf8") as f:
    f.write('\n'.join(arr_corr))

In [6]:
df2 = df
df2 = df2[["incorr_sentence", "corr_sentence"]]
df2.columns = ["source", "gold"]
df2.to_csv("OurDataset_csv.csv", index=False)